In [1]:
!pip install scann

     |████████████████████████████████| 10.9 MB 46 kB/s eta 0:00:01
     |████████████████████████████████| 498.0 MB 3.4 kB/s eta 0:00:01
     |████████████████████████████████| 2.4 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 14.5 MB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 151 kB 2.5 MB/s eta 0:00:01
  Created wheel for termcolor: filename=termcolo

In [2]:
import numpy as np
import h5py
import os
import requests
import tempfile
import time

import scann

2022-06-01 16:27:26.557559: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-01 16:27:26.557588: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
with tempfile.TemporaryDirectory() as tmp:
    response = requests.get("http://ann-benchmarks.com/glove-100-angular.hdf5")
    loc = os.path.join(tmp, "glove.hdf5")
    with open(loc, 'wb') as f:
        f.write(response.content)
    
    glove_h5py = h5py.File(loc, "r")

In [ ]:
list(glove_h5py.keys())

In [ ]:
dataset = glove_h5py['train']
queries = glove_h5py['test']
print(dataset.shape)
print(queries.shape)

In [ ]:
normalized_dataset = dataset / np.linalg.norm(dataset, axis=1)[:, np.newaxis]
# configure ScaNN as a tree - asymmetric hash hybrid with reordering
# anisotropic quantization as described in the paper; see README

# use scann.scann_ops.build() to instead create a TensorFlow-compatible searcher
searcher = scann.scann_ops_pybind.builder(normalized_dataset, 10, "dot_product").  \
           tree(num_leaves=2000, num_leaves_to_search=100, training_sample_size=250000). \
           score_ah(2, anisotropic_quantization_threshold=0.2).reorder(100).build()

In [ ]:
def compute_recall(neighbors, true_neighbors):
    total = 0
    for gt_row, row in zip(true_neighbors, neighbors):
        total += np.intersect1d(gt_row, row).shape[0]
    return total / true_neighbors.size

In [ ]:
# this will search the top 100 of the 2000 leaves, and compute the exact dot products 
# of the top 100 candidates from asymmetric hashing to get the final top 10 candidates.
start = time.time()
neighbors, distances = searcher.search_batched(queries)
end = time.time()

# we are given top 100 neighbors in the ground truth, so select top 10
print("Recall:", compute_recall(neighbors, glove_h5py['neighbors'][:, :10]))
print("Time:", end - start)

In [ ]:
# increasing the leaves to search increases recall at the cost of speed
start = time.time()
neighbors, distances = searcher.search_batched(queries, leaves_to_search=150)
end = time.time()

print("Recall:", compute_recall(neighbors, glove_h5py['neighbors'][:, :10]))
print("Time:", end - start)

In [ ]:
# increasing reordering (the exact scoring of top AH candidates) has a similar effect.
start = time.time()
neighbors, distances = searcher.search_batched(queries, leaves_to_search=150, pre_reorder_num_neighbors=250)
end = time.time()

print("Recall:", compute_recall(neighbors, glove_h5py['neighbors'][:, :10]))
print("Time:", end - start)

In [ ]:
# we can also dynamically configure the number of neighbors returned
# currently returns 10 as configued in ScannBuilder()
neighbors, distances = searcher.search_batched(queries)
print(neighbors.shape, distances.shape)

# now returns 20
neighbors, distances = searcher.search_batched(queries, final_num_neighbors=20)
print(neighbors.shape, distances.shape)

In [ ]:
# we have been exclusively calling batch search so far; the single-query call has the same API
start = time.time()
neighbors, distances = searcher.search(queries[0], final_num_neighbors=5)
end = time.time()

print(neighbors)
print(distances)
print("Latency (ms):", 1000*(end - start))